In [78]:
import json
import os
import functools
import math

import pprint
pp = pprint.PrettyPrinter(indent=4, depth=10, width=120)

In [2]:
recipes = None
with open('/home/tuenut/Downloads/Telegram Desktop/recipes.json', 'r') as file:
    recipes = json.load(file, )

In [3]:
pp.pprint(recipes)

{   'accumulator': {   'category': 'crafting',
                       'energy': 10,
                       'hidden': False,
                       'ingredients': [   {'amount': 2, 'name': 'iron-plate', 'type': 'item'},
                                          {'amount': 5, 'name': 'battery', 'type': 'item'}],
                       'name': 'accumulator',
                       'order': 'e[accumulator]-a[accumulator]',
                       'products': [{'amount': 1, 'name': 'accumulator', 'type': 'item'}]},
    'advanced-circuit': {   'category': 'crafting',
                            'energy': 6,
                            'hidden': False,
                            'ingredients': [   {'amount': 2, 'name': 'plastic-bar', 'type': 'item'},
                                               {'amount': 4, 'name': 'copper-cable', 'type': 'item'},
                                               {'amount': 2, 'name': 'electronic-circuit', 'type': 'item'}],
                            'name':

    'splitter': {   'category': 'crafting',
                    'energy': 1,
                    'hidden': False,
                    'ingredients': [   {'amount': 5, 'name': 'iron-plate', 'type': 'item'},
                                       {'amount': 5, 'name': 'electronic-circuit', 'type': 'item'},
                                       {'amount': 4, 'name': 'transport-belt', 'type': 'item'}],
                    'name': 'splitter',
                    'order': 'c[splitter]-a[splitter]',
                    'products': [{'amount': 1, 'name': 'splitter', 'type': 'item'}]},
    'stack-filter-inserter': {   'category': 'crafting',
                                 'energy': 0.5,
                                 'hidden': False,
                                 'ingredients': [   {'amount': 5, 'name': 'electronic-circuit', 'type': 'item'},
                                                    {'amount': 1, 'name': 'stack-inserter', 'type': 'item'}],
                                 'nam

In [193]:
recipes['copper-plate'], recipes['copper-cable']

({'name': 'copper-plate',
  'category': 'smelting',
  'products': [{'type': 'item', 'name': 'copper-plate', 'amount': 1}],
  'ingredients': [{'type': 'item', 'amount': 1, 'name': 'copper-ore'}],
  'hidden': False,
  'energy': 3.5,
  'order': 'c[copper-plate]'},
 {'name': 'copper-cable',
  'category': 'crafting',
  'products': [{'type': 'item', 'name': 'copper-cable', 'amount': 2}],
  'ingredients': [{'type': 'item', 'amount': 1, 'name': 'copper-plate'}],
  'hidden': False,
  'energy': 0.5,
  'order': 'a[copper-cable]'})

In [4]:
PRODUCT_TYPES = {'advanced-crafting',
 'centrifuging',
 'chemistry',
 'crafting',
 'crafting-with-fluid',
 'oil-processing',
 'rocket-building',
 'smelting'}

In [15]:
producers = {
    "electric-furnace": {
        "name": "electric-furnace", 
        "prod_types": ['smelting'], 
        "speed": 2,
        "mod slots": 2
    },
    "assembling-machine-1": {
        "name": "assembling-machine-1", 
        "prod_types": ['crafting'], 
        "speed": 0.5,
        "mod slots": 0
    },
    "assembling-machine-2": {
        "name": "assembling-machine-2", 
        "prod_types": ['crafting', 'crafting-with-fluid', 'advanced-crafting'], 
        "speed": 0.75,
        "mod slots": 2
    },
    "assembling-machine-3": {
        "name": "assembling-machine-3", 
        "prod_types": ['crafting', 'crafting-with-fluid', 'advanced-crafting'], 
        "speed": 1.25,
        "mod slots": 4
    },
    "oil-refinery.basic": {
        "name": "oil-refinery.basic", 
        "prod_types": ['oil-processing'], 
        "speed": 1,
        "mod slots": 3
    },
    "oil-refinery.advanced": {
        "name": "oil-refinery.advanced", 
        "prod_types": ['oil-processing'], 
        "speed": 1,
        "mod slots": 3
    },
    "oil-refinery.coal-liquefaction": {
        "name": "oil-refinery.coal-liquefaction", 
        "prod_types": ['oil-processing'], 
        "speed": 1,
        "mod slots": 3
    },
    "chemical-plant": {
        "name": "chemical-plant", 
        "prod_types": ['chemistry'], 
        "speed": 1.25,
        "mod slots": 3
    },
}

In [189]:
class Calculator:
    product = None
    product_asms = None
    assemblers_schema = []
    
    EXCLUDE_LIST = ['iron-ore', 'copper-ore', 'coal', 'petroleum-gas', 'stone', 'water']
    
    def input_data(self, product_name=None, amount_per_minute=0, available_assemblers=[]):
        self.assemblers_schema = []
        self.product = recipes[product_name]
        self.product_asms = available_assemblers
        self.target_speed = amount_per_minute / 60

    def calc(self, product_name=None, amount=1, ):
        if not self.product_asms:
            return "Cant craft without assemblers. Self craft not implemented."
        
        product = self.product if not product_name else recipes[product_name]
                
        asm = self._get_asm(self.product['category'])
        product_time = product['energy'] / asm['speed'] * amount
#         if product_name:
#             product_time *= count
        product_speed = 1 / product_time 
        
        count = self.target_speed / product_speed
            
        for item in product['ingredients']:
            if item['name'] not in self.EXCLUDE_LIST:
                print(item['name'])
                self.calc(product_name=item['name'], amount=item['amount'], )

        self.assemblers_schema.append({
            product['name']: {
                'count of producer': count,
                'int count of producer': math.ceil(count),
                'producer': asm['name'],
                'speed': product_speed
            }
        })
        
#         if not product_name:
#             for item in product['ingredients']:
#                 self.assemblers_schema
        
        return self.assemblers_schema
                
    def _get_asm(self, prod_type):
        asm = [producers[asm_name] for asm_name in self.product_asms if prod_type in producers[asm_name]['prod_types']][0]
        return asm

In [190]:
calculator = Calculator()

In [191]:
calculator.input_data(product_name='assembling-machine-1', amount_per_minute=500, available_assemblers=['assembling-machine-3', 'electric-furnace'])

In [192]:
calculator.calc()

iron-plate
iron-gear-wheel
iron-plate
electronic-circuit
iron-plate
copper-cable
copper-plate


[{'iron-plate': {'count of producer': 210.00000000000003,
   'int count of producer': 211,
   'producer': 'assembling-machine-3',
   'speed': 0.03968253968253968}},
 {'iron-plate': {'count of producer': 46.66666666666667,
   'int count of producer': 47,
   'producer': 'assembling-machine-3',
   'speed': 0.17857142857142858}},
 {'iron-gear-wheel': {'count of producer': 16.666666666666668,
   'int count of producer': 17,
   'producer': 'assembling-machine-3',
   'speed': 0.5}},
 {'iron-plate': {'count of producer': 23.333333333333336,
   'int count of producer': 24,
   'producer': 'assembling-machine-3',
   'speed': 0.35714285714285715}},
 {'copper-plate': {'count of producer': 23.333333333333336,
   'int count of producer': 24,
   'producer': 'assembling-machine-3',
   'speed': 0.35714285714285715}},
 {'copper-cable': {'count of producer': 10.000000000000002,
   'int count of producer': 11,
   'producer': 'assembling-machine-3',
   'speed': 0.8333333333333333}},
 {'electronic-circuit': 

In [188]:
4*

SyntaxError: invalid syntax (<ipython-input-188-ae4802a53650>, line 1)

In [129]:
8.75*60

525.0

In [94]:
5/2

2

In [312]:
class Prod:
    _input_raw = 0
    base_consume = None
    base_produce = None
    
    @property
    def raw(self, ):
        return self._input_raw
    
    @raw.setter
    def raw(self, value):
        self._input_raw = value    
    
    @property
    def produce(self, ):
        if self.base_produce is None: return None
        
        mults = {key: self.raw / val for key, val in  self.base_consume.items()}
        mult = min(mults.values())
        return {key: self.base_produce[key] * mult for key in self.base_produce}
    
    def __add__(self, other):
        if not isinstance(other, Prod): return 'Error'
        else:            
            klass = Prod()
            klass.base_consume = {key: val + other.base_consume[key] if key in other.base_consume else val
                for key, val in self.base_consume.items() }

            klass.base_produce = self.produce
            for key, val in other.produce.items():
                try:
                    klass.base_produce[key] += val
                except KeyError:
                    klass.base_produce[key] = val
                    
            klass.raw = 
                    
            return klass
        
    def __str__(self):
        kls_attrs = {key: val for key, val in dict(self.__class__.__dict__).items() if not key.startswith('_')}
        kls_attrs.update(self.__dict__)
        for key, val in kls_attrs.items():
            if 'property' in val.__str__():
                kls_attrs[key] = getattr(self, key)
        return str(kls_attrs)
    
    __repr__ = __str__
        

class Basic(Prod):
    base_produce = dict(heavy = 30, light = 30, petroleum = 40)    
    base_consume = {'oil': 100}
    
class Advanced(Prod):
    base_produce = dict(heavy = 10, light = 45, petroleum = 55)    
    base_consume = {'oil': 100}
    
class HeavyCrack(Prod):
    base_produce = dict(light = 30)
    base_consume = {'heavy': 40}
    
class LightCrack(Prod):    
    base_produce = dict(petroleum = 20)
    base_consume = {'light': 30}
    
oil = 100


In [317]:
a = Advanced()
a.raw = 100
b = HeavyCrack()
b.raw = 40
c = LightCrack()
c.raw = 30

x = a + b


In [318]:
x

{'base_consume': {'oil': 100}, 'base_produce': {'heavy': 10.0, 'light': 75.0, 'petroleum': 55.0}, 'raw': 0, 'produce': {'heavy': 0.0, 'light': 0.0, 'petroleum': 0.0}}

In [223]:
class A:
    attr = 0
    def __init__(self, ):
        self.b = 2
        for name in self.__dict__:
            print(name, getattr(self, name))
A()

b 2


__class__ <class '__main__.A'>
__delattr__ <method-wrapper '__delattr__' of A object at 0x7fb69c3a7470>
__dict__ {}
__dir__ <built-in method __dir__ of A object at 0x7fb69c3a7470>
__doc__ None
__eq__ <method-wrapper '__eq__' of A object at 0x7fb69c3a7470>
__format__ <built-in method __format__ of A object at 0x7fb69c3a7470>
__ge__ <method-wrapper '__ge__' of A object at 0x7fb69c3a7470>
__getattribute__ <method-wrapper '__getattribute__' of A object at 0x7fb69c3a7470>
__gt__ <method-wrapper '__gt__' of A object at 0x7fb69c3a7470>
__hash__ <method-wrapper '__hash__' of A object at 0x7fb69c3a7470>
__init__ <bound method A.__init__ of <__main__.A object at 0x7fb69c3a7470>>
__init_subclass__ <built-in method __init_subclass__ of type object at 0x2a7ffe8>
__le__ <method-wrapper '__le__' of A object at 0x7fb69c3a7470>
__lt__ <method-wrapper '__lt__' of A object at 0x7fb69c3a7470>
__module__ __main__
__ne__ <method-wrapper '__ne__' of A object at 0x7fb69c3a7470>
__new__ <built-in method __new_

In [224]:
{} + {}

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [270]:
[1, 2] + [3]

[1, 2, 3]

In [281]:
tuple({'a': 1, 's':2})

('a', 's')